In [1]:
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

In [ ]:
!pip install stable-baselines3[extra]
!pip install gym

In [ ]:
!pip install tensorflow==2.3.1 keras-rl2

In [ ]:
import gym
import tensorflow as tf

env = gym.make("SpaceInvaders-v0")
print("Observation Space: ", env.observation_space.shape)
print("Observation Space Low: ", env.observation_space.low)
print("Observation Space High: ", env.observation_space.high)
print("Observation Space Sample: ", env.observation_space.sample())

print("Action Space: ", env.action_space.n)
print("Action Space Sample: ", env.action_space.sample())

In [ ]:
env.unwrapped.get_action_meanings()

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [8]:
#for reference
#you can use this formula [(W−K+2P)/S]+1.
#W is the input volume
#K is the Kernel size
#P is the padding
#S is the stride 

In [9]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [24]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n
model = build_model(height, width, channels, actions)

In [23]:
del model

In [ ]:
model.summary()

In [15]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [16]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [26]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [ ]:
dqn.fit(env, nb_steps=100000, visualize=False, verbose=2)

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))